# Team 5 - HW5 - ETL Data Mart HW
Data Warehousing - Fall 21

Team 5: Vince Purcell, Brandon Mondile

In [34]:
import csv
from decimal import Decimal
from os import system
import random
import datetime
from datetime import date
import math
import sqlite3 as lite
from sqlite3 import Error

## Put team 4 and 6 transactions into our database


In [35]:
#team 4 database extraction and insertion

con = lite.connect(r'Grocery.db')
cur = con.cursor()

con2 = lite.connect(r'grocery_group4_fixed.db')
cur2 = con2.cursor()

cur.execute("DROP TABLE IF EXISTS Team4Transactions")
s = 'CREATE TABLE Team4Transactions(date_text varchar(50), customer_number INT, SKU varchar(50), price FLOAT, '
s = s+'left_in_stock INT, cases_bought INT)'
cur.execute(s)

s2 = 'SELECT * FROM sale_data;'
rows = cur2.execute(s2).fetchall()

insertVar = 'INSERT INTO Team4Transactions VALUES (?, ?, ?, ?, ?, ?);'

cur.executemany(insertVar, rows)


con.commit()
con2.close()


In [36]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute("DROP TABLE IF EXISTS Team6Transactions")
s = 'CREATE TABLE Team6Transactions(date_text varchar(50), customer_number INT, SKU varchar(50), price FLOAT, '
s = s+'left_in_stock INT, cases_bought INT)'
cur.execute(s)

insertVar2 = 'INSERT INTO Team6Transactions VALUES (?, ?, ?, ?, ?, ?);'

line_count = 0
with open('Grocery_team6.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')    
    
    for row in csv_reader:
        cur.execute(insertVar2, row[1:7])
        line_count += 1
            
    
        
print(line_count, " rows inserted")

con.commit()
        
con.close()

15560682  rows inserted


## Deliverable 2

Creation of Source Dimension Table and Updates to Product Dimension Table

In [37]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

def strip_uni(s):
      return s.replace(u'\xa0', u' ')

cur.execute("DROP TABLE IF EXISTS ProductDimension")
s = 'CREATE TABLE ProductDimension(ProductKey INT, SKU varchar(8), BasePrice FLOAT, ProductName varchar(50), ProductClassID INT,'
s = s+' Subcategory varchar(50), Category varchar(50), Department varchar(50), ProductFamily varchar(50), Size varchar(10),' 
s = s+'PerCase INT, Manufacturer varchar(50), Supplier varchar(16))'
cur.execute(s)

cur.execute('drop table if exists SourceDimension')
cur.execute('create Table SourceDimension(SKU INT, Source INT)')

product_classes = {}
with open('product_class.csv', 'r') as csvfile:
    for row in csv.DictReader(csvfile):
        product_dict = {
            "product_class_id" : list(row.values())[0], 
            "product_subcategory" : row['product_subcategory'], 
            "product_category" : row['product_category'],
            "product_department" : row['product_department'], 
            "product_family" : row['product_family']
        }
        product_classes[row['product_subcategory']] = product_dict

i = 0
perCase = 12
with open('products_first_working.csv', 'r') as csvfile:
    for row in csv.DictReader(csvfile):
        i = i + 1
        sku = row['SKU']
        source = int(row['Source'])
        size = strip_uni(row['Size'])
        subcat = row['itemType']
        manufacturer = row['Manufacturer']
        productName = row['Product']
        basePrice = row['BasePrice']

        product_class = product_classes[subcat]
        classId = int(product_class["product_class_id"])
        category = product_class["product_category"]
        department = product_class["product_department"]
        family = product_class["product_family"]

        supplier = "Rowan Warehouse"
        if subcat == "Milk":
            supplier = "Rowan Dairy"


        cur.execute("INSERT INTO SourceDimension VALUES (?,?)",
                        (sku, source))

        cur.execute("INSERT INTO ProductDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)",
                        (i, sku, basePrice, productName, classId, subcat, category, department, family, size, perCase, manufacturer, supplier))

        if ((i % 1000)==0):
            print("inserted row ",str(i))
            con.commit()
        


con.commit()
con.close()

inserted row  1000
inserted row  2000


### Deliverable 2a ###
Product Dimension

In [38]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM ProductDimension LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['ProductKey', 'SKU', 'BasePrice', 'ProductName', 'ProductClassID', 'Subcategory', 'Category', 'Department', 'ProductFamily', 'Size', 'PerCase', 'Manufacturer', 'Supplier']
(1, '42081001', 2.49, 'Jambalaya Rice Mix', 57, 'Rice', 'Starchy Foods', 'Starchy Foods', 'Food', '12ï¿½oz', 12, 'Zatarains', 'Rowan Warehouse')
(2, '42082001', 1.79, 'Jambalaya Rice Mix', 57, 'Rice', 'Starchy Foods', 'Starchy Foods', 'Food', '8ï¿½oz', 12, 'Zatarains', 'Rowan Warehouse')
(3, '42083001', 3.99, 'Guacamole Regular', 83, 'Dips', 'Snack Foods', 'Snack Foods', 'Food', '8ï¿½oz', 12, 'Yucatan', 'Rowan Warehouse')
(4, '42084001', 3.99, 'Coffee Original Blend', 90, 'Coffee', 'Hot Beverages', 'Beverages', 'Drink', '12ï¿½oz', 12, 'Yuban', 'Rowan Warehouse')
(5, '42085001', 2.99, 'GoGurt Variety Pack', 6, 'Yogurt', 'Dairy', 'Dairy', 'Food', '8ï¿½ct', 12, 'Yoplait', 'Rowan Warehouse')
(6, '42086001', 2.0, 'Italian Dressing', 48, 'Sauces', 'Baking Goods', 'Baking Goods', 'Food', '16ï¿½oz', 12, 'Wishbone', 'Rowan W

### Deliverable 2b ###
Source Dimension

In [39]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM SourceDimension LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['SKU', 'Source']
(42081001, 2)
(42082001, 2)
(42083001, 2)
(42084001, 2)
(42085001, 1)
(42086001, 2)
(42087001, 2)
(42088001, 2)
(42089001, 2)
(42090001, 1)


## Deliverable 3 

Store Dimension Table

In [40]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists StoreDimension')

s = 'CREATE TABLE StoreDimension('
s += 'StoreKey INT, StoreManager varchar(50), StoreStreetAddr varchar(50), StoreTown varchar(50),'
s += 'StoreZipCode varchar(15), StorePhone varchar(15), StoreState varchar(20))'
cur.execute(s)

key     = 5
manager = 'Vince Purcell'
addr    = '5 Main St.'
town    = 'Moorestown'
zcode   = '08057'
phone   = '8568675309'
state   = 'NJ'

cur.execute("INSERT INTO StoreDimension VALUES (?,?,?,?,?,?,?)", 
    (key, manager, addr, town, zcode, phone, state))

key     = 6
manager = 'David Luco'
addr    = '1286 N Broad St.'
town    = 'Philadelphia'
zcode   = '19130'
phone   = '2157783490'
state   = 'PA'

cur.execute("INSERT INTO StoreDimension VALUES (?,?,?,?,?,?,?)", 
    (key, manager, addr, town, zcode, phone, state))

key     = 4
manager = 'Kevin Malone'
addr    = '16 Main St.'
town    = 'Moorestown'
zcode   = '08057'
phone   = '8562248896'
state   = 'NJ'

cur.execute("INSERT INTO StoreDimension VALUES (?,?,?,?,?,?,?)", 
    (key, manager, addr, town, zcode, phone, state))

con.commit()
con.close()

In [41]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM StoreDimension LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['StoreKey', 'StoreManager', 'StoreStreetAddr', 'StoreTown', 'StoreZipCode', 'StorePhone', 'StoreState']
(5, 'Vince Purcell', '5 Main St.', 'Moorestown', '08057', '8568675309', 'NJ')
(6, 'David Luco', '1286 N Broad St.', 'Philadelphia', '19130', '2157783490', 'PA')
(4, 'Kevin Malone', '16 Main St.', 'Moorestown', '08057', '8562248896', 'NJ')


## Deliverable 4

Date Dimension Table

In [42]:
# Below are lookup dictionaries used during date dimension generation

# Searched '2020 Holidays' on Google and these were what was listed
holidays = {
    date(2020, 1, 1)    : 'New Years Day',
    date(2020, 1, 20)   : 'MLK Day',
    date(2020, 2, 17)   : 'Washingtons Birthday',
    date(2020, 4, 10)   : 'Good Friday',
    date(2020, 5, 25)   : 'Memorial Day',
    date(2020, 7, 3)    : 'Independence Day',
    date(2020, 9, 7)    : 'Labor Day',
    date(2020, 10, 12)  : 'Columbus Day',
    date(2020, 11, 3)   : 'Election Day',
    date(2020, 11, 11)  : 'Veterans Day',
    date(2020, 11, 26)  : 'Thanksgiving',
    date(2020, 12, 25)  : 'Christmas'
}

solstice_and_equinox = {
    date(2020, 3, 21)  : 'Spring',
    date(2020, 6, 21)  : 'Summer',
    date(2020, 9, 23)  : 'Fall',
    date(2020, 12, 22) : 'Winter',
    date(2021, 3, 21)  : 'Spring',
}

months = {
    1  : 'January',
    2  : 'February',
    3  : 'March',
    4  : 'April',
    5  : 'May',
    6  : 'June',
    7  : 'July',
    8  : 'August',
    9  : 'September',
    10 : 'October',
    11 : 'November',
    12 : 'December'
}

fiscal_year_end = date(2020,7,31)

quarters = {
    date(2020, 1, 1)  : 1,
    date(2020, 4, 1)  : 2,
    date(2020, 7, 1)  : 3,
    date(2020, 10, 1) : 4,
    date(2021, 1, 1)  : 1
}

In [43]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists DateDimension')

s = 'CREATE TABLE DateDimension('
s += 'DateKey INT, Date DATETIME, DayNumberInMonth INT, DayNumberInYear INT, WeekNumberInYear INT, MonthNum INT, '
s += 'MonthTxt varchar(20), Quarter INT, Year INT, FiscalYear INT, isHoliday BOOLEAN, isWeekend BOOLEAN, Season varchar(10))'
cur.execute(s)

simulation_start_date = date(2019, 12, 31)
simulation_end_date = date(2020, 12, 31)
current_date = simulation_start_date
simulation_length = 365

key = 0
dayNum = 0
quarter = 1
quarter_index = 0
season = 'Winter'
season_index = 0

fiscalYear = 2019

for iday in range(0, simulation_length):
    if (( (iday+1) % 20)==0):
        print("Working on Day ", iday + 1)
    current_date += datetime.timedelta(1)
    key += 1
    dayNum += 1

    weekend = False
    if current_date.weekday() >= 5:
        weekend = True
    
    if current_date.month == "1" and current_date.day == "1":
        dayNum = 1

    if current_date > fiscal_year_end:
        fiscalYear = 2020
    
    if current_date == list(solstice_and_equinox.keys())[season_index]:
        season = list(solstice_and_equinox.values())[season_index]
        season_index += 1

    if current_date == list(quarters.keys())[quarter_index]:
        quarter = list(quarters.values())[quarter_index]
        quarter_index += 1

    weekNum = current_date.isocalendar()[1]
    monthNum = current_date.month
    monthText = months[monthNum]
    isHoliday = current_date in holidays.keys()

    cur.execute("INSERT INTO DateDimension VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", 
        (key, current_date, current_date.day, dayNum, weekNum, monthNum, monthText, quarter, current_date.year, fiscalYear, isHoliday, weekend, season))

con.commit()
con.close()

Working on Day  20
Working on Day  40
Working on Day  60
Working on Day  80
Working on Day  100
Working on Day  120
Working on Day  140
Working on Day  160
Working on Day  180
Working on Day  200
Working on Day  220
Working on Day  240
Working on Day  260
Working on Day  280
Working on Day  300
Working on Day  320
Working on Day  340
Working on Day  360


In [44]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DateDimension LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'Date', 'DayNumberInMonth', 'DayNumberInYear', 'WeekNumberInYear', 'MonthNum', 'MonthTxt', 'Quarter', 'Year', 'FiscalYear', 'isHoliday', 'isWeekend', 'Season']
(1, '2020-01-01', 1, 1, 1, 1, 'January', 1, 2020, 2019, 1, 0, 'Winter')
(2, '2020-01-02', 2, 2, 1, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(3, '2020-01-03', 3, 3, 1, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(4, '2020-01-04', 4, 4, 1, 1, 'January', 1, 2020, 2019, 0, 1, 'Winter')
(5, '2020-01-05', 5, 5, 1, 1, 'January', 1, 2020, 2019, 0, 1, 'Winter')
(6, '2020-01-06', 6, 6, 2, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(7, '2020-01-07', 7, 7, 2, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(8, '2020-01-08', 8, 8, 2, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(9, '2020-01-09', 9, 9, 2, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')
(10, '2020-01-10', 10, 10, 2, 1, 'January', 1, 2020, 2019, 0, 0, 'Winter')


## Deliverable 6

In [45]:
#create/(drop,recreate) table, insert team 5's data into dailyLevel table

con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists DailyLevelSalesFact')

s = 'CREATE TABLE DailyLevelSalesFact('
s += 'DateKey INT, ProductKey INT, StoreKey INT, NumSoldToday INT, '
s += 'CostOfItemSold FLOAT, SalesTotal FLOAT, GrossProfit FLOAT, SalePrice FLOAT, BasePrice FLOAT)'
cur.execute(s)

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT Transactions2020.*, COUNT(Transactions2020.SKU) AS numberSold, ROUND(SUM(price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM Transactions2020 '
s2 += 'JOIN DateDimension ON DateDimension.Date = Transactions2020.date_text '
s2 += 'GROUP BY date_text, Transactions2020.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, price, numberSold, saletotalSKU, ProductKey, '
s3 += 'ROUND((BasePrice * numberSold), 2) as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)

count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 5
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]

    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

736740  rows inserted


In [46]:
#Do the same thing for team 4

con = lite.connect(r'Grocery.db')
cur = con.cursor()


#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT Team4Transactions.*, COUNT(Team4Transactions.SKU) AS numberSold, ROUND(SUM(price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM Team4Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team4Transactions.date_text '
s2 += 'GROUP BY date_text, Team4Transactions.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, price, numberSold, saletotalSKU, ProductKey, ROUND((BasePrice * numberSold), 2) ' 
s3 += 'as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)

count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 4
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

733497  rows inserted


In [47]:
#Do the same thing for team 4

con = lite.connect(r'Grocery.db')
cur = con.cursor()


#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS TransactView;')
s2  = 'CREATE VIEW TransactView AS '
s2 += 'SELECT Team6Transactions.*, COUNT(Team6Transactions.SKU) AS numberSold, ROUND(SUM(price), 2) AS saletotalSKU, DateKey '
s2 += 'FROM Team6Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team6Transactions.date_text '
s2 += 'GROUP BY date_text, Team6Transactions.SKU;'

cur.execute(s2)

s3 = 'SELECT DateKey, TransactView.SKU, price, numberSold, saletotalSKU, ProductKey, ROUND((BasePrice * numberSold), 2) '
s3 += 'as productBasePriceTotal, BasePrice '
s3 += 'FROM TransactView '
s3 += 'JOIN ProductDimension ON TransactView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY DateKey, TransactView.SKU'

results = cur.execute(s3)

count = 0

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[5]
    StoreKey = 6
    NumSoldToday = row[3]
    CostOfItemSold = row[6]
    SalesTotal = row[4]
    GrossProfit = round((SalesTotal - CostOfItemSold),2)
    SalePrice = row[2]
    BasePrice = row[7]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelSalesFact VALUES (?,?,?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumSoldToday, CostOfItemSold, SalesTotal, 
                                 GrossProfit, SalePrice, BasePrice))

print(count, " rows inserted")

con.commit()
con.close()

735316  rows inserted


### Store 5 Daily Sales

In [48]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelSalesFact WHERE (StoreKey = 5) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumSoldToday', 'CostOfItemSold', 'SalesTotal', 'GrossProfit', 'SalePrice', 'BasePrice']
(1, 1, 5, 25, 62.25, 74.75, 12.5, 2.99, 2.49)
(1, 2, 5, 18, 32.22, 38.7, 6.48, 2.15, 1.79)
(1, 3, 5, 17, 67.83, 81.43, 13.6, 4.79, 3.99)
(1, 4, 5, 23, 91.77, 110.17, 18.4, 4.79, 3.99)
(1, 5, 5, 23, 68.77, 82.57, 13.8, 3.59, 2.99)
(1, 6, 5, 13, 26.0, 31.2, 5.2, 2.4, 2.0)
(1, 7, 5, 20, 231.8, 278.2, 46.4, 13.91, 11.59)
(1, 8, 5, 17, 84.83, 101.83, 17.0, 5.99, 4.99)
(1, 9, 5, 23, 82.57, 99.13, 16.56, 4.31, 3.59)
(1, 10, 5, 23, 110.17, 132.25, 22.08, 5.75, 4.79)


### Store 4 Daily Sales

In [49]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelSalesFact WHERE (StoreKey = 4) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumSoldToday', 'CostOfItemSold', 'SalesTotal', 'GrossProfit', 'SalePrice', 'BasePrice']
(2, 1, 4, 23, 57.27, 0.0, -57.27, '$7.99', 2.49)
(2, 2, 4, 20, 35.8, 0.0, -35.8, '$1.55', 1.79)
(2, 3, 4, 28, 111.72, 0.0, -111.72, '$1.49', 3.99)
(2, 4, 4, 26, 103.74, 0.0, -103.74, '$3.82', 3.99)
(2, 5, 4, 29, 86.71, 0.0, -86.71, '$3.68', 2.99)
(2, 6, 4, 16, 32.0, 0.0, -32.0, '$6.99', 2.0)
(2, 7, 4, 22, 254.98, 0.0, -254.98, '$1.74', 11.59)
(2, 8, 4, 21, 104.79, 0.0, -104.79, '$5.49', 4.99)
(2, 9, 4, 27, 96.93, 0.0, -96.93, '$3.83', 3.59)
(2, 10, 4, 24, 114.96, 0.0, -114.96, '$3.89', 4.79)


### Store 6 Daily Sales

In [50]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelSalesFact WHERE (StoreKey = 6) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumSoldToday', 'CostOfItemSold', 'SalesTotal', 'GrossProfit', 'SalePrice', 'BasePrice']
(2, 1, 6, 18, 44.82, 44.82, 0.0, 2.49, 2.49)
(2, 2, 6, 26, 46.54, 46.54, 0.0, 1.79, 1.79)
(2, 3, 6, 21, 83.79, 83.79, 0.0, 3.99, 3.99)
(2, 4, 6, 27, 107.73, 107.73, 0.0, 3.99, 3.99)
(2, 5, 6, 26, 77.74, 77.74, 0.0, 2.99, 2.99)
(2, 6, 6, 24, 48.0, 48.0, 0.0, 2.0, 2.0)
(2, 7, 6, 17, 197.03, 197.03, 0.0, 11.59, 11.59)
(2, 8, 6, 22, 109.78, 109.78, 0.0, 4.99, 4.99)
(2, 9, 6, 20, 71.8, 71.8, 0.0, 3.59, 3.59)
(2, 10, 6, 28, 134.12, 134.12, 0.0, 4.79, 4.79)


## Deliverable 7

In [51]:
#create/(drop,recreate) table, insert team 5's data into dailyLevel table

con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists DailyLevelInventoryFact')

s = 'CREATE TABLE DailyLevelInventoryFact('
s += 'DateKey INT, ProductKey INT, StoreKey INT, NumAvail INT,'
s += 'CostToStoreItem FLOAT, CostToStoreCase FLOAT, CasesPurchasedToDate INT)'
cur.execute(s)

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventoryView;')
s2  = 'CREATE VIEW InventoryView AS '
s2 += 'SELECT Transactions2020.*, COUNT(SKU) AS numberSold, DateKey '
s2 += 'FROM Transactions2020 '
s2 += 'JOIN DateDimension ON DateDimension.Date = Transactions2020.date_text '
s2 += 'GROUP BY Transactions2020.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT DateKey, ProductKey, InventoryView.SKU, numberSold, '
s3 += 'ROUND((BasePrice * numberSold), 2) as itemCostToStore, left_in_stock, cases_bought, BasePrice '
s3 += 'FROM InventoryView '
s3 += 'JOIN ProductDimension ON InventoryView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY InventoryView.SKU, DateKey'

results = cur.execute(s3)

count = 0

casesTallyTotal = {}

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[1]
    StoreKey = 5
    NumAvail = row[5]
    CostToStoreItem = round(row[4], 2)
    CostToStoreCase = round((row[6]*row[7]*12), 2)
    CasesPurchasedTD = row[6]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelInventoryFact VALUES (?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumAvail, CostToStoreItem, CostToStoreCase, CasesPurchasedTD))

print(count, " rows inserted")

con.commit()
con.close()

736740  rows inserted


In [52]:
#repeate for team 4 data
con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventoryView;')
s2  = 'CREATE VIEW InventoryView AS '
s2 += 'SELECT Team4Transactions.*, COUNT(SKU) AS numberSold, DateKey '
s2 += 'FROM Team4Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team4Transactions.date_text '
s2 += 'GROUP BY Team4Transactions.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT DateKey, ProductKey, InventoryView.SKU, numberSold, '
s3 += 'ROUND((BasePrice * numberSold), 2) as itemCostToStore, left_in_stock, cases_bought, BasePrice '
s3 += 'FROM InventoryView '
s3 += 'JOIN ProductDimension ON InventoryView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY InventoryView.SKU, DateKey'

results = cur.execute(s3)

count = 0

casesTallyTotal = {}

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[1]
    StoreKey = 4
    NumAvail = row[5]
    CostToStoreItem = round(row[4], 2)
    CostToStoreCase = round((row[6]*row[7]*12), 2)
    CasesPurchasedTD = row[6]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelInventoryFact VALUES (?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumAvail, CostToStoreItem, CostToStoreCase, CasesPurchasedTD))

print(count, " rows inserted")

con.commit()
con.close()

733497  rows inserted


In [53]:
#repeat for team 6 data
con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventoryView;')
s2  = 'CREATE VIEW InventoryView AS '
s2 += 'SELECT Team6Transactions.*, COUNT(SKU) AS numberSold, DateKey '
s2 += 'FROM Team6Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team6Transactions.date_text '
s2 += 'GROUP BY Team6Transactions.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT DateKey, ProductKey, InventoryView.SKU, numberSold, '
s3 += 'ROUND((BasePrice * numberSold), 2) as itemCostToStore, left_in_stock, cases_bought, BasePrice '
s3 += 'FROM InventoryView '
s3 += 'JOIN ProductDimension ON InventoryView.SKU = ProductDimension.SKU '
s3 += 'GROUP BY InventoryView.SKU, DateKey'

results = cur.execute(s3)

count = 0

casesTallyTotal = {}

for row in results.fetchall():
    DateKey = row[0]
    ProductKey = row[1]
    StoreKey = 6
    NumAvail = row[5]
    CostToStoreItem = round(row[4], 2)
    CostToStoreCase = round((row[6]*row[7]*12), 2)
    CasesPurchasedTD = row[6]
    
    count += 1
        
    InsertDailyVar = 'INSERT INTO DailyLevelInventoryFact VALUES (?,?,?,?,?,?,?)'

    cur.execute(InsertDailyVar, (DateKey, ProductKey, StoreKey, NumAvail, CostToStoreItem, CostToStoreCase, CasesPurchasedTD))

print(count, " rows inserted")

con.commit()
con.close()

735316  rows inserted


### Store 5 Daily Inventory

In [54]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelInventoryFact WHERE (StoreKey = 5) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumAvail', 'CostToStoreItem', 'CostToStoreCase', 'CasesPurchasedToDate']
(1, 1, 5, 83, 62.25, 209.16, 7)
(2, 1, 5, 82, 74.7, 268.92, 9)
(3, 1, 5, 52, 72.21, 268.92, 9)
(4, 1, 5, 83, 84.66, 418.32, 14)
(5, 1, 5, 49, 67.23, 418.32, 14)
(6, 1, 5, 22, 57.27, 418.32, 14)
(7, 1, 5, 83, 57.27, 627.48, 21)
(8, 1, 5, 60, 59.76, 627.48, 21)
(9, 1, 5, 84, 67.23, 747.0, 25)
(10, 1, 5, 57, 69.72, 747.0, 25)


### Store 4 Daily Inventory

In [55]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelInventoryFact WHERE (StoreKey = 4) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumAvail', 'CostToStoreItem', 'CostToStoreCase', 'CasesPurchasedToDate']
(2, 1, 4, 67, 57.27, 0.0, 0)
(3, 1, 4, 68, 42.33, 0.0, 0)
(4, 1, 4, 51, 34.86, 59.76, 2)
(5, 1, 4, 37, 59.76, 89.64, 3)
(6, 1, 4, 73, 57.27, 0.0, 0)
(7, 1, 4, 50, 47.31, 59.76, 2)
(8, 1, 4, 67, 52.29, 0.0, 0)
(9, 1, 4, 46, 49.8, 59.76, 2)
(10, 1, 4, 74, 37.35, 0.0, 0)
(11, 1, 4, 59, 59.76, 29.88, 1)


### Store 6 Daily Inventory

In [56]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM DailyLevelInventoryFact WHERE (StoreKey = 6) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['DateKey', 'ProductKey', 'StoreKey', 'NumAvail', 'CostToStoreItem', 'CostToStoreCase', 'CasesPurchasedToDate']
(2, 1, 6, 99, 44.82, 0.0, 0)
(3, 1, 6, 81, 57.27, 0.0, 0)
(4, 1, 6, 94, 74.7, 89.64, 3)
(5, 1, 6, 64, 42.33, 89.64, 3)
(6, 1, 6, 47, 54.78, 89.64, 3)
(7, 1, 6, 97, 67.23, 268.92, 9)
(8, 1, 6, 70, 67.23, 268.92, 9)
(9, 1, 6, 103, 49.8, 418.32, 14)
(10, 1, 6, 83, 52.29, 418.32, 14)
(11, 1, 6, 98, 69.72, 507.96, 17)


## Deliverable 8-11

In [57]:
def genQuarterlyFactTable(results, cur, StoreKey):
    count = 0

    CasesTotalYear = {}
    CostTotalYear = {}
    SoldTotalYear = {}
    casesByQuarter = [0,0,0,0,0]
    caseIndex = 0

    for row in results.fetchall():

        Row5 = str(row[5]).replace("$","")
        #Since team 4's data came from CSV they have a $ in most of their prices in the DB (while still being a float)
        Row5 = float(Row5)

        ProductKey = row[0]
        QuarterAndYear = ('Quarter ' + str(row[1]) + ', Year ' +  str(row[2]))
        QuarterVar = row[1]
        YearVar = row[2]

        if int(row[1]) == 1:
            casesByQuarter = [0,0,0,0,0]
        caseIndex = int(row[1])
        NumCasesQuart = row[7] - sum(casesByQuarter)
        casesByQuarter[caseIndex] = NumCasesQuart

        CasesOnHand = round(row[6]/12)
        CostToStoreQuart = round((row[3]*row[4]),2)
        SoldByStoreQuart = round((Row5*row[4]),2)
        
        CasesTotalYear = sum(casesByQuarter)

        if(row[0] not in CostTotalYear.keys()):
            CostTotalYear[row[0]] = round((row[3]*row[4]),2)
        else:
            CostTotalYear[row[0]] += round((row[3]*row[4]),2)
            
        if(row[0] not in SoldTotalYear.keys()):
            SoldTotalYear[row[0]] = round((Row5*row[4]),2)
        else:
            SoldTotalYear[row[0]] += round((Row5*row[4]),2)
        
        count += 1
            
        InsertDailyVar = 'INSERT INTO QuarterInventoryFact VALUES (?,?,?,?,?,?,?,?,?,?,?,?)'

        cur.execute(InsertDailyVar, (ProductKey, StoreKey, QuarterAndYear, QuarterVar, YearVar, CasesTotalYear,
                                    NumCasesQuart, CasesOnHand, CostToStoreQuart, SoldByStoreQuart, 
                                    round(CostTotalYear[row[0]],2), round(SoldTotalYear[row[0]], 2)))
        
    return count

In [58]:
#create/(drop,recreate) table, insert team 5's data into dailyLevel table

con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('drop table if exists QuarterInventoryFact')

s = 'CREATE TABLE QuarterInventoryFact('
s += 'ProductKey INT, StoreKey INT, QuarterAndYear VARCHAR(50), Quarter INT, Year INT, CasesPurchToDate INT, '
s += 'CasesPurchThisQuart INT, CasesOnHand INT, TotalCostThisQuart FLOAT, TotalSoldByStoreThisQuart FLOAT, '
s += 'TotalCostToStoreYTD FLOAT, TotalSoldByStoreYTD FLOAT)'
cur.execute(s)

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventorySubView;')
s2  = 'CREATE VIEW InventorySubView AS '
s2 += 'SELECT Transactions2020.*, COUNT(SKU) AS numberSold, Year, Quarter '
s2 += 'FROM Transactions2020 '
s2 += 'JOIN DateDimension ON DateDimension.Date = Transactions2020.date_text '
s2 += 'GROUP BY Transactions2020.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT ProductDimension.ProductKey, Quarter, Year, BasePrice, '
s3 += 'SUM(numberSold) AS TotalSoldQuart, InventorySubView.price, left_in_stock, MAX(cases_bought) AS CasesThisQuart '
s3 += 'FROM InventorySubView '
s3 += 'JOIN ProductDimension ON ProductDimension.SKU = InventorySubView.SKU '
s3 += 'GROUP BY ProductDimension.ProductKey, Quarter '

results = cur.execute(s3)

count = genQuarterlyFactTable(results, cur, 5)

print(count, " rows inserted")

con.commit()
con.close()

8300  rows inserted


In [59]:
#repeat for team 4

con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventorySubView;')
s2  = 'CREATE VIEW InventorySubView AS '
s2 += 'SELECT Team4Transactions.*, COUNT(SKU) AS numberSold, Year, Quarter '
s2 += 'FROM Team4Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team4Transactions.date_text '
s2 += 'GROUP BY Team4Transactions.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT ProductDimension.ProductKey, Quarter, Year, BasePrice, '
s3 += 'SUM(numberSold) AS TotalSoldQuart, InventorySubView.price, left_in_stock, MAX(cases_bought) AS CasesThisQuart '
s3 += 'FROM InventorySubView '
s3 += 'JOIN ProductDimension ON ProductDimension.SKU = InventorySubView.SKU '
s3 += 'GROUP BY ProductDimension.ProductKey, Quarter '

results = cur.execute(s3)

count = genQuarterlyFactTable(results, cur, 4)

print(count, " rows inserted")

con.commit()
con.close()

8300  rows inserted


In [60]:
#repeat for team 6

con = lite.connect(r'Grocery.db')
cur = con.cursor()

#must create view first for performance reasons (10+min vs 25sec)

cur.execute('DROP VIEW IF EXISTS InventorySubView;')
s2  = 'CREATE VIEW InventorySubView AS '
s2 += 'SELECT Team6Transactions.*, COUNT(SKU) AS numberSold, Year, Quarter '
s2 += 'FROM Team6Transactions '
s2 += 'JOIN DateDimension ON DateDimension.Date = Team6Transactions.date_text '
s2 += 'GROUP BY Team6Transactions.SKU, date_text'

cur.execute(s2)

s3  = 'SELECT ProductDimension.ProductKey, Quarter, Year, BasePrice, '
s3 += 'SUM(numberSold) AS TotalSoldQuart, InventorySubView.price, left_in_stock, MAX(cases_bought) AS CasesThisQuart '
s3 += 'FROM InventorySubView '
s3 += 'JOIN ProductDimension ON ProductDimension.SKU = InventorySubView.SKU '
s3 += 'GROUP BY ProductDimension.ProductKey, Quarter '

results = cur.execute(s3)

count = genQuarterlyFactTable(results, cur, 6)

print(count, " rows inserted")

con.commit()
con.close()

8300  rows inserted


### Store 5 Quarter Inventory Fact

In [61]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM QuarterInventoryFact WHERE (StoreKey=5) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['ProductKey', 'StoreKey', 'QuarterAndYear', 'Quarter', 'Year', 'CasesPurchToDate', 'CasesPurchThisQuart', 'CasesOnHand', 'TotalCostThisQuart', 'TotalSoldByStoreThisQuart', 'TotalCostToStoreYTD', 'TotalSoldByStoreYTD']
(1, 5, 'Quarter 1, Year 2020', 1, 2020, 194, 194, 7, 5629.89, 6760.39, 5629.89, 6760.39)
(1, 5, 'Quarter 2, Year 2020', 2, 2020, 380, 186, 7, 5592.54, 6715.54, 11222.43, 13475.93)
(1, 5, 'Quarter 3, Year 2020', 3, 2020, 570, 190, 6, 5689.65, 6832.15, 16912.08, 20308.08)
(1, 5, 'Quarter 4, Year 2020', 4, 2020, 749, 179, 6, 5383.38, 6464.38, 22295.46, 26772.46)
(2, 5, 'Quarter 1, Year 2020', 1, 2020, 191, 191, 8, 4004.23, 4809.55, 4004.23, 4809.55)
(2, 5, 'Quarter 2, Year 2020', 2, 2020, 380, 189, 7, 4048.98, 4863.3, 8053.21, 9672.85)
(2, 5, 'Quarter 3, Year 2020', 3, 2020, 563, 183, 4, 3986.33, 4788.05, 12039.54, 14460.9)
(2, 5, 'Quarter 4, Year 2020', 4, 2020, 752, 189, 6, 4027.5, 4837.5, 16067.04, 19298.4)
(3, 5, 'Quarter 1, Year 2020', 1, 2020, 190, 190, 7, 8885.73, 10

### Store 4 Quarter Inventory Fact

In [62]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM QuarterInventoryFact WHERE (StoreKey=4) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['ProductKey', 'StoreKey', 'QuarterAndYear', 'Quarter', 'Year', 'CasesPurchToDate', 'CasesPurchThisQuart', 'CasesOnHand', 'TotalCostThisQuart', 'TotalSoldByStoreThisQuart', 'TotalCostToStoreYTD', 'TotalSoldByStoreYTD']
(1, 4, 'Quarter 1, Year 2020', 1, 2020, 5, 5, 1, 4701.12, 5022.08, 4701.12, 5022.08)
(1, 4, 'Quarter 2, Year 2020', 2, 2020, 4, -1, 2, 4708.59, 1172.42, 9409.71, 6194.5)
(1, 4, 'Quarter 3, Year 2020', 3, 2020, 4, 0, 2, 4613.97, 1445.34, 14023.68, 7639.84)
(1, 4, 'Quarter 4, Year 2020', 4, 2020, 5, 1, 1, 4845.54, 11656.54, 18869.22, 19296.38)
(2, 4, 'Quarter 1, Year 2020', 1, 2020, 5, 5, 1, 3476.18, 20371.58, 3476.18, 20371.58)
(2, 4, 'Quarter 2, Year 2020', 2, 2020, 5, 0, 2, 3436.8, 5740.8, 6912.98, 26112.38)
(2, 4, 'Quarter 3, Year 2020', 3, 2020, 5, 0, 2, 3381.31, 6252.59, 10294.29, 32364.97)
(2, 4, 'Quarter 4, Year 2020', 4, 2020, 5, 0, 2, 3511.98, 4963.86, 13806.27, 37328.83)
(3, 4, 'Quarter 1, Year 2020', 1, 2020, 4, 4, 2, 7309.68, 2436.56, 7309.68, 2436.56)
(3, 4, 

### Store 6 Quarter Inventory Fact

In [63]:
con = lite.connect(r'Grocery.db')
cur = con.cursor()

cur.execute('SELECT * FROM QuarterInventoryFact WHERE (StoreKey=6) LIMIT 10')
names = [description[0] for description in cur.description]

print(names)
rows = cur.fetchall()
for row in rows:
    print(row)

con.commit()
con.close()

['ProductKey', 'StoreKey', 'QuarterAndYear', 'Quarter', 'Year', 'CasesPurchToDate', 'CasesPurchThisQuart', 'CasesOnHand', 'TotalCostThisQuart', 'TotalSoldByStoreThisQuart', 'TotalCostToStoreYTD', 'TotalSoldByStoreYTD']
(1, 6, 'Quarter 1, Year 2020', 1, 2020, 177, 177, 8, 5341.05, 5341.05, 5341.05, 5341.05)
(1, 6, 'Quarter 2, Year 2020', 2, 2020, 365, 188, 8, 5624.91, 5624.91, 10965.96, 10965.96)
(1, 6, 'Quarter 3, Year 2020', 3, 2020, 551, 186, 7, 5585.07, 5585.07, 16551.03, 16551.03)
(1, 6, 'Quarter 4, Year 2020', 4, 2020, 727, 176, 7, 5256.39, 5256.39, 21807.42, 21807.42)
(2, 6, 'Quarter 1, Year 2020', 1, 2020, 183, 183, 8, 3977.38, 3977.38, 3977.38, 3977.38)
(2, 6, 'Quarter 2, Year 2020', 2, 2020, 369, 186, 8, 4000.65, 4000.65, 7978.03, 7978.03)
(2, 6, 'Quarter 3, Year 2020', 3, 2020, 551, 182, 6, 3945.16, 3945.16, 11923.19, 11923.19)
(2, 6, 'Quarter 4, Year 2020', 4, 2020, 737, 186, 7, 3975.59, 3975.59, 15898.78, 15898.78)
(3, 6, 'Quarter 1, Year 2020', 1, 2020, 182, 182, 8, 8797.9